# Modeling and Simulations Project 1

Studio 1

Jinfay Yuan & Jack Mao

In [16]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *

from pandas import read_html

In [19]:
tables = read_html('https://en.wikipedia.org/wiki/United_States_immigration_statistics',
header=0,
index_col=0)

[Empty DataFrame
 Columns: [This article needs to be updated. Please update this article to reflect recent events or newly available information. (November 2010)]
 Index: [],
                                        2000      1990      1980     1970  \
 Region and country or area                                                 
 Total                              31107889  19767316  14079906  9619302   
 Reported by region and/or country  31107573  18959158  13192563  9303570   
 Europe                              4915557   4350403   5149572  5740891   
 Northern and Western Europe         2070446   2058853   2384257  2629200   
 Northern Europe                      974619    968271   1083499  1271591   
 British Isles                        834225    809972    866966   937474   
 United Kingdom (total)               677715    640145    669149   686099   
 Great Britain (total)                677751    623614    649318   645262   
 England                              423609    405588 